Google Cloud authentication and data download

In [7]:
! gcloud auth login
! gcloud auth application-default login
! gcloud config set project dl4h-final-project-383605

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=mA4KTl8iW7ExFCgiSIGV20WPRGNlqP&access_type=offline&code_challenge=xuy5gjuf9GwNlmobryR0GgFOlCXznHfpqWf2kC4oOvM&code_challenge_method=S256


You are now logged in as [camerongreenwalt@gmail.com].
Your current project is [dl4h-final-project-383605].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID
Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_i

In [12]:
! pip install --upgrade google-api-python-client google-cloud-storage
from google.cloud import storage
import gzip
import shutil
import os

# Replace these values with your project and bucket as needed
project_id = "dl4h-final-project-383605"
mimic3_bucket = "mimiciii-1.4.physionet.org"

storage_client = storage.Client(project=project_id)
bucket = storage_client.bucket(mimic3_bucket)
data_folder = "data"
for blob in bucket.list_blobs():
    if "CHARTEVENTS" in blob.name:
        continue

    gz_path = f"{data_folder}/{blob.name}"
    blob.download_to_filename(gz_path)

    with gzip.open(gz_path, 'rb') as f_in:
        with open(gz_path.replace(".gz", ""), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    os.remove(gz_path)

# # Extract all the files
# ! gunzip {data_folder}/*.gz


[notice] A new release of pip available: 22.3.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
! pip uninstall torch torchvision torchaudio

^C


Library imports and data preparation

In [47]:
! pip install pyhealth
from pyhealth.datasets import MIMIC3Dataset, SampleDataset
from pyhealth.data import Patient, Visit, Event
import pandas as pd
from pyhealth.datasets import split_by_patient, get_dataloader
from pyhealth.models import BaseModel
from pyhealth.trainer import Trainer
from pyhealth.metrics.binary import binary_metrics_fn
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List, Dict, Optional
from enum import Enum
from functools import reduce
from operator import mul
from tqdm import tqdm

# Set this to the directory with all MIMIC-3 dataset files
data_root = "data"


[notice] A new release of pip available: 22.3.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [113]:
# Load the dataset

mimic3_ds = MIMIC3Dataset(
        root=data_root,
        tables=["DIAGNOSES_ICD", "PROCEDURES_ICD"],
        dev=False
)

In [114]:
# Print dataset statistics

mimic3_ds.stat()


Statistics of base dataset (dev=False):
	- Dataset: MIMIC3Dataset
	- Number of patients: 46520
	- Number of visits: 58976
	- Number of visits per patient: 1.2678
	- Number of events per visit in DIAGNOSES_ICD: 11.0384
	- Number of events per visit in PROCEDURES_ICD: 4.0711



'\nStatistics of base dataset (dev=False):\n\t- Dataset: MIMIC3Dataset\n\t- Number of patients: 46520\n\t- Number of visits: 58976\n\t- Number of visits per patient: 1.2678\n\t- Number of events per visit in DIAGNOSES_ICD: 11.0384\n\t- Number of events per visit in PROCEDURES_ICD: 4.0711\n'

In [115]:
# Find all diagnoses codes
# Remove diagnoses codes with fewer than 5 occurences in the dataset

all_diagnosis_codes = []
for patient_id, patient in mimic3_ds.patients.items():
  for i in range(len(patient)):
    visit: Visit = patient[i]
    conditions = visit.get_code_list(table="DIAGNOSES_ICD")
    all_diagnosis_codes.extend(conditions)

codes = pd.Series(all_diagnosis_codes)
diag_code_counts = codes.value_counts()
filtered_diag_codes = diag_code_counts[diag_code_counts > 4].index.values
num_unique_diag_codes = len(filtered_diag_codes)

In [117]:
MIN_NUM_VISITS_PER_PATIENT = 2

filtered_patients = {}
for patient_id, patient in tqdm(mimic3_ds.patients.items()):

    filtered_patient: Patient = Patient(
        patient_id=patient.patient_id,
        birth_datetime=patient.birth_datetime,
        death_datetime=patient.death_datetime,
        gender=patient.gender,
        ethnicity=patient.ethnicity
    )

    for i_visit, visit in enumerate(patient):
        filtered_visit: Visit = Visit(
            visit_id=visit.visit_id,
            patient_id=visit.patient_id,
            encounter_time=visit.encounter_time,
            discharge_time=visit.discharge_time,
            discharge_status=visit.discharge_status
        )

        diagnoses_codes = visit.get_code_list("DIAGNOSES_ICD")
        procedures_codes = visit.get_code_list("PROCEDURES_ICD")

        if len(diagnoses_codes) > 0:
            diagnosis_events = visit.event_list_dict["DIAGNOSES_ICD"]
            for i_event in range(len(diagnosis_events) - 1, -1, -1):
                event: Event = diagnosis_events[i_event]
                if event.code not in filtered_diag_codes:
                    diagnosis_events.pop(i_event) # Remove the diagnosis code with fewer than the cutoff occurrences

            if len(diagnosis_events) == 0: continue # Don't include visits with no diagnoses

            filtered_visit.set_event_list("DIAGNOSES_ICD", diagnosis_events)
        else:
            continue # Don't include visits with no diagnoses

        if len(procedures_codes) > 0:
           filtered_visit.set_event_list("PROCEDURES_ICD", visit.event_list_dict["PROCEDURES_ICD"])

        filtered_patient.add_visit(filtered_visit)

    if len(filtered_patient.visits) >= MIN_NUM_VISITS_PER_PATIENT:
        filtered_patients[patient_id] = filtered_patient


100%|██████████| 46520/46520 [01:15<00:00, 619.28it/s]


In [118]:
mimic3_ds.patients = filtered_patients
mimic3_ds.stat()


Statistics of base dataset (dev=False):
	- Dataset: MIMIC3Dataset
	- Number of patients: 7496
	- Number of visits: 19905
	- Number of visits per patient: 2.6554
	- Number of events per visit in DIAGNOSES_ICD: 12.9735
	- Number of events per visit in PROCEDURES_ICD: 4.0975



'\nStatistics of base dataset (dev=False):\n\t- Dataset: MIMIC3Dataset\n\t- Number of patients: 7496\n\t- Number of visits: 19905\n\t- Number of visits per patient: 2.6554\n\t- Number of events per visit in DIAGNOSES_ICD: 12.9735\n\t- Number of events per visit in PROCEDURES_ICD: 4.0975\n'

In [123]:
# Define the tasks

DIAGNOSES_KEY = "conditions"
PROCEDURES_KEY = "procedures"
INTERVAL_DAYS_KEY = "days_since_first_visit"

def flatten(l: List):
    return [item for sublist in l for item in sublist]

def patient_level_readmission_prediction(patient, time_window: int = 30, max_length_visits: int = None):
    """
    patient is a <pyhealth.data.Patient> object
    """
    samples = []

    # # if the patient only has one visit, we drop it
    # if len(patient) <= 1:
    #     return []

    sorted_visits = sorted(patient, key=lambda visit: visit.encounter_time)

    # Clip the patient visits to the most recent max_length_visits + 1 if max_length_visits is not None
    if max_length_visits is not None:
        n_visits = len(sorted_visits)
        if n_visits > max_length_visits + 1:
            sorted_visits = sorted_visits[n_visits - (max_length_visits + 1):]

    feature_visits: List[Visit] = sorted_visits[:-1]
    last_visit: Visit = sorted_visits[-1]
    second_to_last_visit: Visit = feature_visits[-1]
    first_visit: Visit = feature_visits[0]

    # step 1: define label
    time_diff = (last_visit.encounter_time - second_to_last_visit.encounter_time).days
    readmission_label = 1 if time_diff <= time_window else 0

    # step 2: obtain features
    visits_diagnoses = []
    visits_procedures = []
    visits_intervals = []
    for idx, visit in enumerate(feature_visits):
        diagnoses = visit.get_code_list(table="DIAGNOSES_ICD")
        procedures = visit.get_code_list(table="PROCEDURES_ICD")
        time_diff_from_first_visit = (visit.encounter_time - first_visit.encounter_time).days

        if len(diagnoses) == 0:
            continue

        visits_diagnoses.append(diagnoses)
        visits_procedures.append(procedures)
        visits_intervals.append([str(time_diff_from_first_visit)])

    unique_diagnoses = list(set(flatten(visits_diagnoses)))

    # step 3: exclusion criteria
    if len(unique_diagnoses) == 0:
        return []

    # step 4: assemble the sample
    samples.append(
        {
            "patient_id": patient.patient_id,
            "visit_id": visit.visit_id,
            "diagnoses": visits_diagnoses,
            "procedures": visits_procedures,
            "intervals": visits_intervals,
            "label": readmission_label,
        }
    )
    return samples

In [124]:
# Create the task datasets
mimic3_dxtx = mimic3_ds.set_task(task_fn=patient_level_readmission_prediction)
print(len(mimic3_dxtx))

Generating samples for patient_level_readmission_prediction: 100%|██████████| 7496/7496 [00:00<00:00, 46850.81it/s]


7496


In [125]:
BATCH_SIZE = 32
train, val, test = split_by_patient(mimic3_dxtx, [0.8, 0.1, 0.1])

train_loader = get_dataloader(train, batch_size=BATCH_SIZE, shuffle=True)
val_loader = get_dataloader(val, batch_size=BATCH_SIZE, shuffle=False)
test_loader = get_dataloader(test, batch_size=BATCH_SIZE, shuffle=False)

In [126]:
# Define the models
VERY_BIG_NUMBER = 1e30
VERY_SMALL_NUMBER = 1e-30
VERY_POSITIVE_NUMBER = VERY_BIG_NUMBER
VERY_NEGATIVE_NUMBER = -VERY_BIG_NUMBER

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

class MaskDirection(Enum):
    FORWARD = 'forward'
    BACKWARD = 'backward'
    DIAGONAL = 'diagonal'
    NONE = 'none'

class MaskedLayerNorm(nn.Module):
    def __init__(self, normalized_shape: int):
        super().__init__()
        self.scale = nn.parameter.Parameter(torch.ones(normalized_shape, dtype=torch.float32, device=device))
        self.bias = nn.parameter.Parameter(torch.zeros(normalized_shape, dtype=torch.float32, device=device))
        self.normalized_shape = normalized_shape

    def forward(self, x: torch.Tensor, eps=1e-5):
        mean = torch.mean(x, dim=-1, keepdim=True)
        variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
        norm_x = (x - mean) * torch.rsqrt(variance + eps)
        return norm_x * self.scale + self.bias

class Flatten(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x: torch.Tensor, keep: int):
        fixed_shape = list(x.size())
        start = len(fixed_shape) - keep
        left = reduce(mul, [fixed_shape[i] or x.shape[i] for i in range(start)])
        out_shape = [left] + [fixed_shape[i] or x.shape[i] for i in range(start, len(fixed_shape))]
        return torch.reshape(x, out_shape)


class Unflatten(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, v: torch.Tensor, ref: torch.Tensor, embedding_dim):
        batch_size = ref.shape[0]
        n_visits = ref.shape[1]
        out = torch.reshape(v, [batch_size, n_visits, embedding_dim])
        return out


class AttentionPooling(nn.Module):
    def __init__(self, embedding_size: int):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(embedding_size, embedding_size),
            nn.ReLU(),
            nn.Linear(embedding_size, embedding_size)
        )

    def forward(self, inputs):
        x, mask = inputs
        x = self.fc(x)
        x[mask] = VERY_NEGATIVE_NUMBER
        soft = F.softmax(x, dim=1)
        x[mask] = 0
        attn_output = torch.sum(soft * x, 1)
        return attn_output


class MultiHeadAttention(nn.Module):
    def __init__(self, direction, dropout, num_units, num_heads=4):
        super().__init__()
        self.num_heads = num_heads
        self.direction = direction
        self.dropout = dropout
        self.num_units = num_units
        self.q_linear = nn.Linear(num_units, num_units, bias=False)
        self.k_linear = nn.Linear(num_units, num_units, bias=False)
        self.v_linear = nn.Linear(num_units, num_units, bias=False)

    def forward(self, inputs):

        # because of self-attention, queries and keys is equal to inputs
        input_tensor, input_mask = inputs
        queries = input_tensor
        keys = input_tensor

        # Linear projections
        Q = self.q_linear(queries)  # (N, L_q, d)
        K = self.k_linear(keys)  # (N, L_k, d)
        V = self.v_linear(keys)  # (N, L_k, d)

        # print('Q shape: ', Q.get_shape())

        # Split and concat
        assert self.num_units % self.num_heads == 0
        Q_ = torch.cat(torch.split(Q, self.num_units // self.num_heads, dim=2), dim=0)  # (h*N, L_q, d/h)
        K_ = torch.cat(torch.split(K, self.num_units // self.num_heads, dim=2), dim=0)  # (h*N, L_k, d/h)
        V_ = torch.cat(torch.split(V, self.num_units // self.num_heads, dim=2), dim=0)  # (h*N, L_k, d/h)

        # Multiplication
        outputs = torch.matmul(Q_, torch.permute(K_, [0, 2, 1]))  # (h*N, L_q, L_k)

        # Scale
        outputs = outputs / (list(K_.shape)[-1] ** 0.5)  # (h*N, L_q, L_k)

        # Key Masking
        key_masks = torch.sign(torch.sum(torch.abs(K_), dim=-1))  # (h*N, T_k)
        key_masks = torch.unsqueeze(key_masks, 1)  # (h*N, 1, T_k)
        key_masks = torch.tile(key_masks, [1, list(Q_.shape)[1], 1])  # (h*N, T_q, T_k)

        # Apply masks to outputs
        paddings = torch.ones_like(outputs, device=device) * (-2 ** 32 + 1)  # exp mask
        outputs = torch.where(key_masks == 0, paddings, outputs)  # (h*N, T_q, T_k)

        n_visits = list(input_tensor.shape)[1]
        sw_indices = torch.arange(0, n_visits, dtype=torch.int32, device=device)
        sw_col, sw_row = torch.meshgrid(sw_indices, sw_indices)
        if self.direction == MaskDirection.DIAGONAL:
            # shape of (n_visits, n_visits)
            attention_mask = (torch.diag(- torch.ones([n_visits], dtype=torch.int32, device=device)) + 1).bool()
        elif self.direction == MaskDirection.FORWARD:
            attention_mask = torch.greater(sw_row, sw_col)  # shape of (n_visits, n_visits)
        else: # MaskDirection.BACKWARD
            attention_mask = torch.greater(sw_col, sw_row)  # shape of (n_visits, n_visits)
        adder = (1.0 - attention_mask.type(outputs.dtype)) * -10000.0
        outputs += adder

        # softmax
        outputs = F.softmax(outputs, -1)  # (h*N, T_q, T_k)

        # Query Masking
        query_masks = torch.sign(torch.sum(torch.abs(Q_), dim=-1))  # (h*N, T_q)
        query_masks = torch.unsqueeze(query_masks, -1)  # (h*N, T_q, 1)
        query_masks = torch.tile(query_masks, [1, 1, list(K_.shape)[1]])  # (h*N, T_q, T_k)

        # Apply masks to outputs
        outputs = outputs * query_masks

        # Dropouts
        outputs = F.dropout(outputs, p=self.dropout)
        # Weighted sum
        outputs = torch.matmul(outputs, V_)  # ( h*N, T_q, C/h)

        # Restore shape
        outputs = torch.cat(torch.split(outputs, outputs.shape[0] // self.num_heads, dim=0), dim=2)  # (N, L_q, d)

        # input padding
        val_mask = torch.unsqueeze(input_mask, -1)
        outputs = torch.multiply(outputs, (~val_mask).float())

        return outputs


class MaskEnc(nn.Module):
    def __init__(
            self,
            embedding_dim: int,
            num_heads: int,
            dropout: float = 0.1,
            batch_first: bool = True,
            temporal_mask_direction: MaskDirection = MaskDirection.NONE,
    ):
        super().__init__()

        self.embedding_dim = embedding_dim
        self.temporal_mask_direction = temporal_mask_direction

        self.attention = MultiHeadAttention(
            direction=temporal_mask_direction,
            dropout=dropout,
            num_units=embedding_dim,
            num_heads=num_heads
        )

        self.fc = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(embedding_dim, embedding_dim)
        )

        self.layer_norm1 = MaskedLayerNorm(embedding_dim)
        self.layer_norm2 = MaskedLayerNorm(embedding_dim)

    def forward(self, inputs):
        x, key_padding_mask = inputs

        attn_output = self.attention((x, key_padding_mask))
        attn_output = self.layer_norm1(x + attn_output)
        out = self.fc(attn_output)
        out = out * (~key_padding_mask.unsqueeze(-1)).float()
        out = self.layer_norm2(out + attn_output)
        out = out * (~key_padding_mask.unsqueeze(-1)).float()

        return out, key_padding_mask

    def _make_temporal_mask(self, n: int) -> Optional[torch.Tensor]:
        if self.temporal_mask_direction == MaskDirection.NONE:
            return None
        if self.temporal_mask_direction == MaskDirection.FORWARD:
            return torch.tril(torch.full((n, n), -10000, device=device)).fill_diagonal_(0).float()
        if self.temporal_mask_direction == MaskDirection.BACKWARD:
            return torch.triu(torch.full((n, n), -10000, device=device)).fill_diagonal_(0).float()
        if self.temporal_mask_direction == MaskDirection.DIAGONAL:
            return torch.zeros(n, n, device=device).fill_diagonal_(-10000).float()


class BiteNet(nn.Module):
    def __init__(
            self,
            embedding_dim: int = 128,
            num_heads: int = 4,
            dropout: float = 0.1,
            batch_first: bool = True,
            n_mask_enc_layers: int = 2,
            use_procedures: bool = True,
            use_intervals: bool = True,
    ):
        super().__init__()

        self.use_intervals = use_intervals
        self.use_procedures = use_procedures
        self.embedding_dim = embedding_dim

        self.flatten = Flatten()
        self.unflatten = Unflatten()

        def _make_mask_enc_block(temporal_mask_direction: MaskDirection = MaskDirection.NONE):
            return MaskEnc(
                embedding_dim = embedding_dim,
                num_heads = num_heads,
                dropout = dropout,
                batch_first = batch_first,
                temporal_mask_direction = temporal_mask_direction,
            )

        self.code_attn = nn.Sequential()
        self.visit_attn_fw = nn.Sequential()
        self.visit_attn_bw = nn.Sequential()
        for _ in range(n_mask_enc_layers):
            self.code_attn.append(_make_mask_enc_block(MaskDirection.DIAGONAL))
            self.visit_attn_fw.append(_make_mask_enc_block(MaskDirection.FORWARD))
            self.visit_attn_bw.append(_make_mask_enc_block(MaskDirection.BACKWARD))

        # Attention pooling layers
        self.code_attn.append(AttentionPooling(embedding_dim))
        self.visit_attn_fw.append(AttentionPooling(embedding_dim))
        self.visit_attn_bw.append(AttentionPooling(embedding_dim))

        self.fc = nn.Sequential(
            nn.Linear(2*embedding_dim, embedding_dim),
            nn.ReLU()
        )

    def forward(
            self,
            embedded_codes: torch.Tensor,
            embedded_intervals: torch.Tensor,
            codes_mask: torch.Tensor,
            visits_mask: torch.Tensor,
    ) -> torch.Tensor:

        codes_mask = ~(codes_mask.bool())

        # input tensor, reshape 4 dimension to 3
        flattened_codes = self.flatten(embedded_codes, 2)

        # input mask, reshape 3 dimension to 2
        flattened_codes_mask = self.flatten(codes_mask, 1)

        code_attn = self.code_attn((flattened_codes, flattened_codes_mask))
        code_attn = self.unflatten(code_attn, embedded_codes, self.embedding_dim)

        if self.use_intervals:
            code_attn += embedded_intervals

        visits_mask = ~(visits_mask.bool())

        u_fw = self.visit_attn_fw((code_attn, visits_mask))
        u_bw = self.visit_attn_bw((code_attn, visits_mask))
        u_bi = torch.cat([u_fw, u_bw], dim=-1)

        s = self.fc(u_bi)
        return s

class PyHealthBiteNet(BaseModel):
    def __init__(
            self,
            dataset: SampleDataset,
            feature_keys: List[str],
            label_key: str,
            mode: str,
            embedding_dim: int = 128,
            n_mask_enc_layers: int = 2,
            use_intervals: bool = True,
            use_procedures: bool = True,
            num_heads: int = 4,
            dropout: float = 0.1,
            batch_first: bool = True,
            **kwargs
    ):
        super().__init__(dataset, feature_keys, label_key, mode)

        self.use_intervals = use_intervals
        self.use_procedures = use_procedures

        # Any BaseModel should have these attributes, as functions like add_feature_transform_layer uses them
        self.feat_tokenizers = {}
        self.embeddings = nn.ModuleDict()
        self.linear_layers = nn.ModuleDict()
        self.label_tokenizer = self.get_label_tokenizer()
        self.embedding_dim = embedding_dim

        # self.add_feature_transform_layer will create a transformation layer for each feature
        for feature_key in self.feature_keys:
            input_info = self.dataset.input_info[feature_key]
            self.add_feature_transform_layer(
                feature_key, input_info, special_tokens=["<pad>", "<unk>"]
            )

        # final output layer
        output_size = self.get_output_size(self.label_tokenizer)
        self.bite_net = BiteNet(
            embedding_dim = embedding_dim,
            num_heads = num_heads,
            dropout = dropout,
            batch_first = batch_first,
            use_intervals=use_intervals,
            use_procedures=use_procedures,
            n_mask_enc_layers=n_mask_enc_layers
        )

        self.fc = nn.Linear(self.embedding_dim, output_size)

    def forward(self, **kwargs) -> Dict[str, torch.Tensor]:

        embeddings = {}
        masks = {}
        for feature_key in self.feature_keys:
            input_info = self.dataset.input_info[feature_key]

            # each patient's feature is represented by [[code1, code2],[code3]]
            assert input_info["dim"] == 3 and input_info["type"] == str
            feature_vals = kwargs[feature_key]

            x = self.feat_tokenizers[feature_key].batch_encode_3d(feature_vals, truncation=(False, False))
            x = torch.tensor(x, dtype=torch.long, device=self.device)
            pad_idx = self.feat_tokenizers[feature_key].vocabulary("<pad>")
            #create the mask
            mask = (x != pad_idx).long()
            embeds = self.embeddings[feature_key](x)
            embeddings[feature_key] = embeds
            masks[feature_key] = mask

        embedded_codes = embeddings['diagnoses']
        codes_mask = masks['diagnoses']
        if self.use_procedures:
            embedded_codes = torch.cat((embedded_codes, embeddings['procedures']), dim=2)
            codes_mask = torch.cat((codes_mask, masks['procedures']), dim=2)

        output = self.bite_net(embedded_codes, embeddings['intervals'].squeeze(2), codes_mask, masks['intervals'].squeeze(-1))
        logits = self.fc(output)

        # obtain y_true, loss, y_prob
        y_true = self.prepare_labels(kwargs[self.label_key], self.label_tokenizer)
        loss = self.get_loss_function()(logits, y_true)
        y_prob = self.prepare_y_prob(logits)

        return {"loss": loss, "y_prob": y_prob, "y_true": y_true}

model_dxtx = PyHealthBiteNet(
    dataset = mimic3_dxtx,
    feature_keys = ['procedures', 'diagnoses', 'intervals'],
    label_key = "label",
    mode = "binary",
    embedding_dim=128
).to(device)

data = next(iter(train_loader))
model_dxtx(**data)

Device: cuda


{'loss': tensor(0.7000, device='cuda:0',
        grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 'y_prob': tensor([[0.5118],
         [0.5014],
         [0.5201],
         [0.5056],
         [0.4996],
         [0.5080],
         [0.5105],
         [0.5045],
         [0.5048],
         [0.5111],
         [0.5119],
         [0.5054],
         [0.5109],
         [0.5024],
         [0.5124],
         [0.5054],
         [0.5067],
         [0.5089],
         [0.5090],
         [0.5099],
         [0.5103],
         [0.5112],
         [0.5072],
         [0.5097],
         [0.5046],
         [0.5080],
         [0.5018],
         [0.5092],
         [0.5121],
         [0.4986],
         [0.5023],
         [0.5077]], device='cuda:0', grad_fn=<SigmoidBackward0>),
 'y_true': tensor([[0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [1.],
         [0.],
         [0.],
         [0.],
         [1.],
 

In [127]:
# Initialize model and train
model_dxtx = PyHealthBiteNet(
    dataset = mimic3_dxtx,
    feature_keys = ["diagnoses", "procedures", "intervals"],
    label_key = "label",
    mode = "binary",
    use_intervals=False
).to(device)

trainer_dxtx = Trainer(model=model_dxtx, device=device)
trainer_dxtx.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=10,
    monitor="pr_auc",
    optimizer_class=torch.optim.RMSprop,
    optimizer_params = {"lr" : 0.0001}
)

PyHealthBiteNet(
  (embeddings): ModuleDict(
    (diagnoses): Embedding(3437, 128, padding_idx=0)
    (procedures): Embedding(1362, 128, padding_idx=0)
    (intervals): Embedding(1925, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (bite_net): BiteNet(
    (flatten): Flatten()
    (unflatten): Unflatten()
    (code_attn): Sequential(
      (0): MaskEnc(
        (attention): MultiHeadAttention(
          (q_linear): Linear(in_features=128, out_features=128, bias=False)
          (k_linear): Linear(in_features=128, out_features=128, bias=False)
          (v_linear): Linear(in_features=128, out_features=128, bias=False)
        )
        (fc): Sequential(
          (0): Linear(in_features=128, out_features=128, bias=True)
          (1): ReLU()
          (2): Dropout(p=0.1, inplace=False)
          (3): Linear(in_features=128, out_features=128, bias=True)
        )
        (layer_norm1): MaskedLayerNorm()
        (layer_norm2): MaskedLayerNorm()
      )
      (1): MaskEnc(
     

Epoch 0 / 10:   0%|          | 0/188 [00:00<?, ?it/s]

--- Train epoch-0, step-188 ---
loss: 0.4936
Evaluation: 100%|██████████| 24/24 [00:00<00:00, 63.66it/s]
--- Eval epoch-0, step-188 ---
pr_auc: 0.3863
roc_auc: 0.5802
f1: 0.2116
loss: 0.4903
New best pr_auc score (0.3863) at epoch-0, step-188



Epoch 1 / 10:   0%|          | 0/188 [00:00<?, ?it/s]

--- Train epoch-1, step-376 ---
loss: 0.4768
Evaluation: 100%|██████████| 24/24 [00:00<00:00, 60.30it/s]
--- Eval epoch-1, step-376 ---
pr_auc: 0.3879
roc_auc: 0.5928
f1: 0.2255
loss: 0.4993
New best pr_auc score (0.3879) at epoch-1, step-376



...

--- Train epoch-7, step-1504 ---
loss: 0.4133
Evaluation: 100%|██████████| 24/24 [00:00<00:00, 61.35it/s]
--- Eval epoch-7, step-1504 ---
pr_auc: 0.3858
roc_auc: 0.5998
f1: 0.3438
loss: 0.6530



Epoch 8 / 10:   0%|          | 0/188 [00:00<?, ?it/s]

--- Train epoch-8, step-1692 ---
loss: 0.3970
Evaluation: 100%|██████████| 24/24 [00:00<00:00, 61.54it/s]
--- Eval epoch-8, step-1692 ---
pr_auc: 0.3923
roc_auc: 0.6109
f1: 0.2815
loss: 0.5923



Epoch 9 / 10:   0%|          | 0/188 [00:00<?, ?it/s]

--- Train epoch-9, step-1880 ---
loss: 0.3815
Evaluation: 100%|██████████| 24/24 [00:00<00:00, 62.66it/s]
--- Eval epoch-9, step-1880 ---
pr_auc: 0.3952
roc_auc: 0.6106
f1: 0.3179
loss: 0.5989
Loaded best model


In [128]:
# option 1: use our built-in evaluation metric
score_dxtx = trainer_dxtx.evaluate(test_loader)
print (score_dxtx)

# option 2: use our pyhealth.metrics to evaluate
y_true_dxtx, y_prob_dxtx, loss_dxtx = trainer_dxtx.inference(test_loader)
binary_metrics_fn(y_true_dxtx, y_prob_dxtx, metrics=["pr_auc"])

Evaluation: 100%|██████████| 24/24 [00:00<00:00, 38.04it/s]


{'pr_auc': 0.3316655664465687, 'roc_auc': 0.5941990802835793, 'f1': 0.18867924528301885, 'loss': 0.46841003745794296}


Evaluation: 100%|██████████| 24/24 [00:00<00:00, 62.83it/s]


{'pr_auc': 0.33439426046529186}

In [103]:
data = next(iter(train_loader))
model_dxtx(**data)

{'loss': tensor(0.4559, device='cuda:0',
        grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 'y_prob': tensor([[0.1593],
         [0.0360],
         [0.2327],
         [0.2694],
         [0.3649],
         [0.1299],
         [0.0518],
         [0.0243],
         [0.0540],
         [0.0545],
         [0.1940],
         [0.1263],
         [0.0421],
         [0.0909],
         [0.3075],
         [0.0950],
         [0.2574],
         [0.2021],
         [0.0538],
         [0.0631],
         [0.0881],
         [0.2980],
         [0.2668],
         [0.2148],
         [0.0890],
         [0.2316],
         [0.1003],
         [0.1232],
         [0.0977],
         [0.1383],
         [0.0947],
         [0.0673]], device='cuda:0', grad_fn=<SigmoidBackward0>),
 'y_true': tensor([[0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
 